In [1]:
from pandas import read_csv, cut
from numpy import array, unique, log2, inf, append, where, square
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import time
from pprint import pprint

In [45]:
file_path = '../catalog4/cat4.csv'
df = read_csv(file_path)
df.head()
y = array(df['class'])
# df = df.rename(columns={"class": "label"})
if 'cat2.csv' in file_path:
    df.drop("Unnamed: 0.1", axis=1, inplace=True)
df.drop(["Unnamed: 0", "galex_objid", "sdss_objid", "spectrometric_redshift", "pred"], axis=1, inplace=True)

In [38]:
df['class'].value_counts()

1    23389
0    10074
Name: class, dtype: int64

In [ ]:
def resample(df, n_samples, replace=True, random_state=None):
    temp = DataFrame()
    length_df = len(df)
    base_size = 10
    if replace == False:
        assert n_samples <= length_df
        return df.sample(n=n_samples, random_state=random_state)
    else:
        while n_samples > base_size:
            temp = temp.append(df.sample(n=base_size), ignore_index=True)
            n_samples -= 10
        temp = temp.append(df.sample(n=n_samples))
    return temp

In [39]:
# code to resample
df_0 = df[df['class'] == 0]
df_1 = df[df['class'] == 1]
new_df_0 = resample(df_0, n_samples=2*len(df_0), random_state=42)
#new_df_1 = resample(df_1, n_samples=300, replace=False, random_state=42)

new_df = new_df_0.append(df_1)

In [21]:
df.head()

,u,g,r,i,z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,...,g-z,r-i,r-z,i-z,fuv-nuv,fuv-u,fuv-g,fuv-r,fuv-i,fuv-z
0,18.286291,16.968243,16.334318,16.039885,15.857203,0.094921,0.073962,0.051166,0.038022,0.028281,...,1.111040,0.294434,0.477116,0.182682,-2.119261,-4.368679,-5.686728,-6.320652,-6.615086,-6.797768
1,22.734962,21.516233,20.178501,19.238029,18.809675,0.105131,0.081917,0.056670,0.042112,0.031323,...,2.706558,0.940472,1.368826,0.428354,-0.371609,-0.231091,-1.449820,-2.787552,-3.728024,-4.156378
2,21.223398,21.143623,21.149973,20.998989,21.145853,0.085433,0.066569,0.046052,0.034222,0.025455,...,-0.002230,0.150984,0.004120,-0.146864,-1.134020,-2.778496,-2.858271,-2.851921,-3.002905,-2.856041
3,21.771444,21.636995,21.564974,21.476393,21.266085,0.074157,0.057783,0.039974,0.029705,0.022095,...,0.370911,0.088581,0.298889,0.210308,-0.945484,-1.074581,-1.209030,-1.281052,-1.369633,-1.579941
4,22.767988,22.018591,21.658953,21.098751,20.734152,0.050161,0.039085,0.027039,0.020093,0.014945,...,1.284439,0.560202,0.924801,0.364599,-0.822910,-1.516697,-2.266094,-2.625732,-3.185934,-3.550533


In [22]:
new_df.head()

,u,g,r,i,z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,...,g-z,r-i,r-z,i-z,fuv-nuv,fuv-u,fuv-g,fuv-r,fuv-i,fuv-z
529,22.550404,21.257240,21.478666,21.499123,21.688509,0.034370,0.026781,0.018527,0.013768,0.010241,...,-0.431269,-0.020456,-0.209843,-0.189386,0.160799,-0.864923,-2.158087,-1.936661,-1.916204,-1.726818
629,20.109095,19.691797,19.591894,19.643299,19.559032,0.060723,0.047315,0.032732,0.024324,0.018092,...,0.132765,-0.051405,0.032862,0.084267,-0.672718,-1.711824,-2.129122,-2.229025,-2.177620,-2.261887
392,20.855820,20.461594,20.716227,20.925772,20.845173,0.064471,0.050235,0.034753,0.025825,0.019209,...,-0.383579,-0.209545,-0.128946,0.080599,0.168247,0.023479,-0.370747,-0.116114,0.093431,0.012833
209,22.424471,21.302855,21.066275,21.105309,20.676207,0.063531,0.049503,0.034246,0.025448,0.018929,...,0.626648,-0.039034,0.390068,0.429102,0.382706,0.293287,-0.828329,-1.064909,-1.025875,-1.454977
564,19.777691,19.726589,20.117905,20.405279,20.943363,0.105370,0.082104,0.056799,0.042208,0.031395,...,-1.216774,-0.287374,-0.825459,-0.538084,0.011721,0.365753,0.314651,0.705967,0.993341,1.531425


In [46]:
def bucketize(dataframe, col_headers, bucket_size):
    assert len(col_headers) == len(bucket_size)
    no_of_columns = len(col_headers)
    for col in range(no_of_columns):
        labels = array([(x + 1) for x in range(bucket_size[col])])
        temp = cut(dataframe[col_headers[col]], bucket_size[col], labels=labels)
        dataframe.drop(col_headers[col], inplace=True, axis=1)
        dataframe[col_headers[col]] = temp
    return dataframe

y = array(df['class'])
df.drop('class', inplace=True, axis=1)
temp = bucketize(df, df.columns, [7 for x in range(len(df.columns))])

In [7]:
df.columns

Index(['u', 'g', 'r', 'i', 'z', 'extinction_u', 'extinction_g', 'extinction_r',
       'extinction_i', 'extinction_z', 'nuv_mag', 'fuv_mag', 'class', 'nuv-u',
       'nuv-g', 'nuv-r', 'nuv-i', 'nuv-z', 'u-g', 'u-r', 'u-i', 'u-z', 'g-r',
       'g-i', 'g-z', 'r-i', 'r-z', 'i-z', 'fuv-nuv', 'fuv-u', 'fuv-g', 'fuv-r',
       'fuv-i', 'fuv-z'],
      dtype='object')

In [8]:
new_df.columns

Index(['u', 'g', 'r', 'i', 'z', 'extinction_u', 'extinction_g', 'extinction_r',
       'extinction_i', 'extinction_z', 'nuv_mag', 'fuv_mag', 'nuv-u', 'nuv-g',
       'nuv-r', 'nuv-i', 'nuv-z', 'u-g', 'u-r', 'u-i', 'u-z', 'g-r', 'g-i',
       'g-z', 'r-i', 'r-z', 'i-z', 'fuv-nuv', 'fuv-u', 'fuv-g', 'fuv-r',
       'fuv-i', 'fuv-z'],
      dtype='object')

In [47]:
X = array(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [48]:
class DecisionTree:
    
    def __init__(self, max_depth=5, min_samples=2):
        self.counter = 0
        self.max_depth = max_depth
        self.min_samples = min_samples
        
    def check_purity(self, y):
        if len(unique(y)) == 1:
            return True
        else:
            return False
        
    def classify_data(self, y):
        unique_classes, counts_unique_classes = unique(y, return_counts=True)

        index = where(counts_unique_classes == max(counts_unique_classes))[0][0]
        classification = unique_classes[index]

        return classification
    
    def get_potential_splits(self, X):
    
        potential_splits = {}
        n_columns = len(X[0])
        for column_index in range(n_columns):  
            potential_splits[column_index] = []
            values = X[:, column_index]
            unique_values = unique(values)

            for index in range(1, len(unique_values)):
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2

                potential_splits[column_index].append(potential_split)

        return potential_splits
    
    def split_data(self, X, y, split_column, split_value):
    
        no_of_columns = len(X[0]) + 1
        split_column_values = X[:, split_column]
        data = append(X, y.reshape(len(X), 1), axis=1)
        data_below = data[data[:, split_column] < split_value]
        data_above = data[data[:, split_column] >= split_value]

        return data_below, data_above
    
    def calculate_entropy(self, label_column):

        _, counts = unique(label_column, return_counts=True)
        probabilities = counts / counts.sum()
        return 1 - sum(square(probabilities))
    
    def calculate_overall_entropy(self, data_below, data_above):
    
        n = len(data_below) + len(data_above)
        p_data_below = len(data_below) / n
        p_data_above = len(data_above) / n

        overall_entropy =  (p_data_below * self.calculate_entropy(data_below[:, -1]) 
                          + p_data_above * self.calculate_entropy(data_above[:, -1]))

        return overall_entropy
    
    def determine_best_split(self, X, y, potential_splits):    
        overall_entropy = inf
        for column_index in potential_splits:
            for value in potential_splits[column_index]:
                data_below, data_above = self.split_data(X, y, split_column=column_index, split_value=value)
                current_overall_entropy = self.calculate_overall_entropy(data_below, data_above)

                if current_overall_entropy <= overall_entropy:
                    overall_entropy = current_overall_entropy
                    best_split_column = column_index
                    best_split_value = value

        return best_split_column, best_split_value
    
    def build_tree(self, X, y):
        if (self.check_purity(y)) or (len(X) < self.min_samples) or (self.counter == self.max_depth):
            classification = self.classify_data(y)

            return classification


        # recursive part
        else:    
            self.counter += 1

            # helper functions 
            potential_splits = self.get_potential_splits(X)
            split_column, split_value = self.determine_best_split(X, y, potential_splits)
            data_below, data_above = self.split_data(X, y, split_column, split_value)

            question = "column_{} <= {}".format(split_column, split_value)
            sub_tree = {question: []}

            # find answers (recursion)
            yes_answer = self.build_tree(data_below[:, :-1], data_below[:, -1])
            no_answer = self.build_tree(data_above[:, :-1], data_above[:, -1])

            # If the answers are the same, then there is no point in asking the qestion.
            # This could happen when the data is classified even though it is not pure
            # yet (min_samples or max_depth base case).
            if yes_answer == no_answer:
                sub_tree = yes_answer
            else:
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)

            return sub_tree
    
    def fit(self, X, y):
        
        start_time = time.time()
        tree = self.build_tree(X, y)
        end_time = time.time()
        print("Time taken to construct the decision tree =", end_time - start_time)
        return tree
    
    def classify_example(self, example, tree):
        question = list(tree.keys())[0]
        feature_name, comparison_operator, value = question.split(" ")
        x = int(feature_name.split("_")[1])

        # ask question
        if example[x] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]

        # when at leaf node
        if type(answer) != dict:
            return answer

        # when at an internal node
        else:
            residual_tree = answer
            return self.classify_example(example, residual_tree)
    
    def predict(self, X_test, tree):
        predictions = array([])
        for example in X_test:
            predictions = append(predictions, self.classify_example(example, tree))

        return predictions

In [51]:
tree = DecisionTree(max_depth=5)

decision_tree = tree.fit(X_train, y_train)
predictions = tree.predict(X_test, decision_tree)

Time taken to construct the decision tree = 10.009169816970825


In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.54      0.53      0.54      3039
           1       0.80      0.81      0.80      7000

    accuracy                           0.72     10039
   macro avg       0.67      0.67      0.67     10039
weighted avg       0.72      0.72      0.72     10039

